In [1]:
import os

import numpy as np 
import pandas as pd

# load the databases
# note that specifying the encoding here was important to getting them to load properly
base_dir = '/Users/dcard/Dropbox/CMU/ARK/terrorism/databases/'
gtd = pd.read_csv(os.path.join(base_dir, 'gtd', 'globalterrorismdb_0617dist.csv'), header=0, encoding='ISO-8859-1', low_memory=False)
rand = pd.read_csv(os.path.join(base_dir, 'RAND', 'RAND_Database_of_Worldwide_Terrorism_Incidents.csv'), header=0, encoding='ISO-8859-1', low_memory=False)


In [2]:
# look at a snapshot of the GTD database
gtd.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [45]:
# display all of the columns in the GTD database
print(gtd.loc[0])

eventid                              197000000001
iyear                                        1970
imonth                                          7
iday                                            2
approxdate                                    NaN
extended                                        0
resolution                                    NaN
country                                        58
country_txt                    Dominican Republic
region                                          2
region_txt            Central America & Caribbean
provstate                                     NaN
city                                Santo Domingo
latitude                                  18.4568
longitude                                -69.9512
specificity                                     1
vicinity                                        0
location                                      NaN
summary                                       NaN
crit1                                           1


In [47]:
# look at a snapshot of the RAND database
rand.head()

,Date,City,Country,Perpetrator,Weapon,Injuries,Fatalities,Description
0,1-May-73,Kabul,Afghanistan,Black December,Firearms,0,0,AFGHANISTAN. A member of the Pakistani Black ...
1,14-Feb-79,Kabul,Afghanistan,Other,Firearms,0,1,AFGHANISTAN. The U.S. Ambassador to Afghanis...
2,7-Sep-79,Kabul,Afghanistan,Unknown,Firearms,0,6,AFGHANISTAN. Six West Germans were shot and k...
3,8-Sep-79,Kabul,Afghanistan,Unknown,Unknown,1,0,"AFGHANISTAN. Lt. Col. Michael Cavanaugh, a U...."
4,9-Sep-79,NaN,Afghanistan,Other,Firearms,1,2,AFGHANISTAN. Afghanistan rebels ambushed a to...


In [48]:
# convert the dates in the RAND database to year, month, day
from datetime import datetime
datetimes = [pd.to_datetime(d) for d in rand['Date']]
years = [d.year for d in datetimes]
months = np.array([d.month for d in datetimes], dtype=int)
days = np.array([d.day for d in datetimes], dtype=int)
rand['iyear'] = years
rand['imonth'] = months
rand['iday'] = days
rand.head()

,Date,City,Country,Perpetrator,Weapon,Injuries,Fatalities,Description,iyear,imonth,iday
0,1-May-73,Kabul,Afghanistan,Black December,Firearms,0,0,AFGHANISTAN. A member of the Pakistani Black ...,1973,5,1
1,14-Feb-79,Kabul,Afghanistan,Other,Firearms,0,1,AFGHANISTAN. The U.S. Ambassador to Afghanis...,1979,2,14
2,7-Sep-79,Kabul,Afghanistan,Unknown,Firearms,0,6,AFGHANISTAN. Six West Germans were shot and k...,1979,9,7
3,8-Sep-79,Kabul,Afghanistan,Unknown,Unknown,1,0,"AFGHANISTAN. Lt. Col. Michael Cavanaugh, a U....",1979,9,8
4,9-Sep-79,NaN,Afghanistan,Other,Firearms,1,2,AFGHANISTAN. Afghanistan rebels ambushed a to...,1979,9,9


In [50]:
# take a subset of events from GTD that are specified as taking place in the United States
gtd_us = gtd[gtd['country_txt'] == 'United States']
gtd_us.shape


(2758, 135)

In [51]:
# do the same for the RAND database
rand_us = rand[rand['Country'] == 'United States']
rand_us.shape


(567, 11)

In [70]:
# create a new database to hold the merged records
df_merged = pd.DataFrame(columns=['iyear', 'imonth', 'iday', 'City', 'Country', 'rand', 'gtd', 'n_matches'])
count = 0
# look through every event in the RAND database
for i in rand_us.index:
    # extract the year, month, day, city, and description
    year = rand_us.loc[i, 'iyear']
    month = rand_us.loc[i, 'imonth']
    day = rand_us.loc[i, 'iday']
    city = str(rand_us.loc[i, 'City'])
    rand_desc = str(rand_us.loc[i, 'Description'])
    # exclude those that don't specify a city
    if city != 'nan':
        # correct for naming conventions
        if city == 'Washington, D.C.' or city == 'Washington, DC':
            city = 'Washington'
        # look for records in GTD with the same year, month, day, and city
        matches = gtd_us[(gtd_us['iyear'] == year) & (gtd_us['imonth'] == month) & (gtd_us['iday'] == day) & (gtd_us['city'] == city)]
        # if there is at least one match, store the first one, and note how many matches there were
        if len(matches) > 0:
            n_matches = len(matches.index)
            first_match = matches.index[0]
            gtd_desc = matches.loc[first_match, 'summary']
            # add the merged record to the new DataFrame
            df_merged.loc[count] = [year, month, day, city, 'United States', rand_desc, gtd_desc, n_matches]
            count += 1
    

In [71]:
# see how many merged events we found
df_merged.shape

(323, 8)

In [73]:
# look at the distribution of the number of matches 
np.bincount(np.array(df_merged['n_matches'], dtype=int))


array([  0, 254,  44,   5,  18,   2])

In [ ]:
# most (254) are matching to a unique event in GTD

In [63]:
# print a list of city values in the GTD to look for weird names
set(gtd_us['city'].values)

{'Afton',
 'Aguada',
 'Aguadilla',
 'Aibonito',
 'Akron',
 'Alameda',
 'Alamogordo',
 'Albany',
 'Albertville',
 'Albion',
 'Albuquerque',
 'Alexandria',
 'Alhambra',
 'Altadena',
 'Alton',
 'Amarillo',
 'Ames',
 'Amherst',
 'Anaheim',
 'Anamosa',
 'Ann Arbor',
 'Annapolis',
 'Appleton',
 'Aptos',
 'Arcata',
 'Arecibo',
 'Arivaca',
 'Arlington',
 'Asbury',
 'Asheville',
 'Ashland',
 'Ashville',
 'Astoria',
 'Athens',
 'Atherton',
 'Atlanta',
 'Auburn',
 'Augusta',
 'Aurora',
 'Austin',
 'Axton',
 "Bailey's Crossroads",
 'Bakersfield',
 'Baltimore',
 'Baraboo',
 'Barnegat',
 'Batavia',
 'Baton Rouge',
 'Baxter Springs',
 'Bayamon',
 'Beaumont',
 'Bedford Park',
 'Bel Air',
 'Bellevue',
 'Bellflower',
 'Bellingham',
 'Belmont',
 'Ben Lomond',
 'Bend',
 'Berkeley',
 'Berlin',
 'Bessemer',
 'Bethesda',
 'Beverly Hills',
 'Biggsville',
 'Billings',
 'Birmingham',
 'Blooming Grove',
 'Bloomington',
 'Bluefield',
 'Boca Raton',
 'Boise',
 'Bonners Ferry',
 'Borinquen',
 'Boston',
 'Boulder',


In [64]:
# do the same for RAND
set(rand_us['City'].values)

{'Albany',
 'Albertville',
 'Albuquerque',
 'Alhambra',
 'Anaheim',
 'Astoria',
 'Atlanta',
 'Auburn',
 'Austin',
 'Baltimore',
 'Bellingham',
 'Berkeley',
 'Bethesda',
 'Birmingham',
 'Bloomington',
 'Boca Raton',
 'Boston',
 'Cambridge',
 'Charlottesville',
 'Chicago',
 'Chico',
 'Clatskanie',
 'Coral Gables',
 'De Kalb',
 'Dearborn',
 'Detroit',
 'East Lansing',
 'Elizabeth',
 'Ellettsville',
 'Emeryville',
 'Erie',
 'Estacada',
 'Eugene',
 'Falls Church Virginia',
 'Fargo',
 'Fort Lauderdale',
 'Fresno',
 'Ft. Hood',
 'Galveston',
 'Garden Grove',
 'Girard',
 'Glen Cove',
 'Glendale',
 'Granada Hills',
 'Grangeville, Idaho',
 'Greenville',
 'Hagerstown',
 'Hialeah',
 'Hoboken',
 'Hollywood',
 'Houghton, Michigan',
 'Houston',
 'Irvine',
 'Jacksonville',
 'Key Biscayne',
 'Key West',
 'Laguna Woods',
 'Langley',
 'Leavenworth',
 'Lincoln',
 'Little Rock',
 'Long Island',
 'Los Angeles',
 'Martiny',
 'Medford',
 'Miami',
 'Middle Island',
 'Miller Place',
 'Minneapolis',
 'Monmouth',

In [3]:
# from a quick glance, only Washington DC needs extra care